# Lap Telemetry Exploration

This notebook analyzes telemetry data collected from hotlaps in Assetto Corsa, providing detailed insights into driving performance and car behavior.

**Session Configuration:**
* **Track**: Autodromo Nazionale Monza
* **Car**: Aston Martin AMR24 (2024 Formula 1)
* **Session Type**: Hotlap Analysis

**Analysis Features:**
* Lap-by-lap telemetry breakdown with normalized distances
* Interactive visualizations with hover data
* Comprehensive driving input analysis (throttle, brake, steering)
* Performance metrics and sector analysis

---

## 1. Dependencies and Configuration

In [21]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
from pathlib import Path
import os
from IPython.display import display, HTML

---

## 2. Data Loading and Preprocessing

The following cell implements intelligent telemetry file discovery and data normalization:

**Automatic File Discovery:**
- Searches multiple common locations for telemetry CSV files
- Automatically selects the most recent file based on creation time
- Supports various project directory structures

**Data Normalization:**
- Standardizes column names across different CSV formats and versions
- Handles missing columns with appropriate defaults
- Creates distance measurements for analysis (relative or absolute)
- Ensures data type consistency for numerical analysis

In [22]:
# Automatically find the most recent telemetry CSV file
import glob
import os

# Try different possible locations for telemetry files
possible_paths = [
    "TELEMETRY/*.csv",           # if running from project root
    "../TELEMETRY/*.csv",        # if running from subdirectory (like data-analytics)
    "../../TELEMETRY/*.csv",     # if running from deeper subdirectory
    "*.csv"                      # current directory
]

telemetry_file = None
for pattern in possible_paths:
    files = glob.glob(pattern)
    if files:
        # Get the most recent file
        telemetry_file = max(files, key=os.path.getctime)
        break

if telemetry_file is None:
    print("No telemetry CSV files found. Please check the file path.")
    print("Current working directory:", os.getcwd())
    print("Looking for files in these patterns:", possible_paths)
    raise FileNotFoundError("No telemetry CSV files found")

print(f"Loading telemetry file: {telemetry_file}")
df = pd.read_csv(telemetry_file)

# Normalize / common rename (adapt to your CSV)
col_map = {}
# common variants -> normalized names used below
if 'Speed (km/h)' in df.columns:
    col_map['Speed (km/h)'] = 'Speed_kmh'
if 'Speed_kmh' not in df.columns and 'Speed' in df.columns:
    col_map['Speed'] = 'Speed_kmh'
if 'iCurrentTime' in df.columns and 'iCurrentTime_ms' not in df.columns:
    col_map['iCurrentTime'] = 'iCurrentTime_ms'
if 'iLastTime' in df.columns and 'iLastTime_ms' not in df.columns:
    col_map['iLastTime'] = 'iLastTime_ms'
if 'iBestTime' in df.columns and 'iBestTime_ms' not in df.columns:
    col_map['iBestTime'] = 'iBestTime_ms'
if 'CarX' in df.columns and 'X' not in df.columns:
    col_map['CarX'] = 'X'
if 'CarY' in df.columns and 'Y' not in df.columns:
    col_map['CarY'] = 'Y'
if 'CarZ' in df.columns and 'Z' not in df.columns:
    col_map['CarZ'] = 'Z'

df = df.rename(columns=col_map)

# Ensure essential columns exist; set defaults if missing
if 'Completed Laps' in df.columns:
    df = df.rename(columns={'Completed Laps': 'CompletedLaps'})
elif 'CompletedLaps' not in df.columns:
    raise KeyError("CSV must contain 'CompletedLaps' or 'Completed Laps' column")

if 'Speed_kmh' not in df.columns:
    df['Speed_kmh'] = np.nan
if 'Throttle' not in df.columns:
    df['Throttle'] = np.nan
if 'Brake' not in df.columns:
    df['Brake'] = np.nan
if 'Steering' not in df.columns:
    # maybe steerAngle
    for candidate in ['steerAngle','SteeringAngle','Steering']:
        if candidate in df.columns:
            df = df.rename(columns={candidate:'Steering'})
            break
    else:
        df['Steering'] = np.nan

# PRIORITIZE normalized distance over absolute distance
# Use Distance column (normalized) if available, otherwise fallback to absolute
if 'Distance' in df.columns:
    # Distance column is already normalized (0-based per lap)
    print("✅ Using normalized distance column (Distance)")
    pass  # Keep Distance as is
elif 'DistanceTraveled_m' in df.columns:
    # Fallback to absolute distance if normalized not available
    df['Distance'] = df['DistanceTraveled_m']
    print("⚠️  Using absolute distance (DistanceTraveled_m) as fallback")
else:
    # Last resort: create index-based distance
    df['Distance'] = np.arange(len(df))
    print("⚠️  Using index-based distance as fallback")

# Convert CompletedLaps to int for grouping
df['CompletedLaps'] = df['CompletedLaps'].fillna(0).astype(int)

print("Columns after normalization:", list(df.columns))
print("Rows:", len(df))
print("Data range:")
print(f"  - Laps: {df['CompletedLaps'].min()} to {df['CompletedLaps'].max()}")
print(f"  - Speed: {df['Speed_kmh'].min():.1f} to {df['Speed_kmh'].max():.1f} km/h")
print(f"  - Distance: {df['Distance'].min():.1f} to {df['Distance'].max():.1f} m")
print(f"  - First few rows:")
display(df.head())

Loading telemetry file: ../TELEMETRY\telemetry_2025-09-13_16-18-26.csv
✅ Using normalized distance column (Distance)
Columns after normalization: ['Timestamp', 'Speed_kmh', 'RPM', 'Throttle', 'Brake', 'Steering', 'Gear', 'CompletedLaps', 'iCurrentTime_ms', 'CurrentLapTime_str', 'iLastTime_ms', 'iBestTime_ms', 'DistanceTraveled_m', 'LapNumberTotal', 'CurrentSectorIndex', 'LastSectorTime_ms', 'IsInPit', 'IsInPitLane', 'TyreCompound', 'X', 'Y', 'Z', 'Flag', 'SurfaceGrip', 'Distance', 'DistanceTraveled_m_Original']
Rows: 2021
Data range:
  - Laps: 0 to 2
  - Speed: 77.0 to 335.0 km/h
  - Distance: 0.0 to 6431.5 m
  - First few rows:


,Timestamp,Speed_kmh,RPM,Throttle,Brake,Steering,Gear,CompletedLaps,iCurrentTime_ms,CurrentLapTime_str,...,IsInPit,IsInPitLane,TyreCompound,X,Y,Z,Flag,SurfaceGrip,Distance,DistanceTraveled_m_Original
0,1757772902,165.27,12109,1.0,0.0,0.100,3,0,19563,0:19.563,...,False,False,Soft (S),28.941195,-10.632852,805.717224,0,1.0,0.000,146.824
1,1757772903,166.17,10337,1.0,0.0,0.101,4,0,19668,0:19.668,...,False,False,Soft (S),27.939253,-10.650772,810.441833,0,1.0,4.868,151.692
2,1757772903,167.06,10470,1.0,0.0,0.121,4,0,19770,0:19.770,...,False,False,Soft (S),26.861551,-10.675842,814.895752,0,1.0,9.469,156.293
3,1757772903,167.91,10503,1.0,0.0,0.131,4,0,19869,0:19.869,...,False,False,Soft (S),25.606434,-10.715822,819.472473,0,1.0,14.069,160.893
4,1757772903,168.38,10510,1.0,0.0,0.136,4,0,19971,0:19.971,...,False,False,Soft (S),24.190029,-10.766002,824.021362,0,1.0,18.775,165.599


---

## 3. Lap Data Extraction and Helper Functions

In [23]:
# Helper function for lap data extraction and lap list preparation

def rows_for_lap(df, lap_number: int) -> pd.DataFrame:
    """
    Extract telemetry data for a specific lap from the complete dataset.
    
    AC's lap counting logic: data recorded DURING lap K has CompletedLaps == K-1.
    This is because CompletedLaps represents the number of laps already finished,
    not the current lap being driven.
    
    Args:
        df (pd.DataFrame): Complete telemetry dataset
        lap_number (int): Target lap number (1-based, e.g., 1 for first lap)
        
    Returns:
        pd.DataFrame: Filtered dataset containing only data from the specified lap
        
    Example:
        - To get data from lap 1: look for CompletedLaps == 0
        - To get data from lap 2: look for CompletedLaps == 1
        - etc.
    """
    target = lap_number - 1
    return df[df['CompletedLaps'] == target].reset_index(drop=True)

# Build available laps list based on data range
max_completed = int(df['CompletedLaps'].max())
available_laps = list(range(1, max_completed + 1))
print(f"Available laps for analysis: {available_laps}")
print(f"Total laps detected: {len(available_laps)}")


Available laps for analysis: [1, 2]
Total laps detected: 2


## 4. Comprehensive Telemetry Dashboard (All Laps)

In [24]:
# Interactive telemetry visualization dashboard using Plotly
# Creates comprehensive 2x3 grid layout for each lap with all key parameters

for lap in available_laps:
    lap_df = rows_for_lap(df, lap)
    if lap_df.empty:
        continue

    # Create comprehensive subplot layout: 2 rows × 3 columns
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=('Gear vs Distance', 'Speed vs Distance', 'RPM vs Distance',
                       'Throttle vs Distance', 'Brake vs Distance', 'Steering vs Distance'),
        vertical_spacing=0.15,
        horizontal_spacing=0.08
    )

    # Modern color palette for consistent visual identity
    colors = {
        'gear': '#FF6B35',      # Orange-red for gear changes
        'speed': '#004E89',     # Deep blue for speed profile
        'rpm': '#FF9F1C',       # Golden orange for engine RPM
        'throttle': '#2E8B57',  # Sea green for throttle input
        'brake': '#DC143C',     # Crimson for brake input
        'steering': '#8A2BE2'   # Blue violet for steering input
    }

    # Row 1, Column 1: Gear Progression Analysis
    if 'Gear' in lap_df.columns and not lap_df['Gear'].isna().all():
        fig.add_trace(
            go.Scatter(
                x=lap_df['Distance'], 
                y=lap_df['Gear'],
                mode='lines',
                line=dict(color=colors['gear'], width=3, shape='hv'),  # Step-like for gear changes
                name='Gear',
                hovertemplate='Distance: %{x:.1f}m<br>Gear: %{y}<extra></extra>'
            ),
            row=1, col=1
        )
    else:
        fig.add_annotation(
            x=0.5, y=0.5, xref='x domain', yref='y domain',
            text="No Gear Data Available", showarrow=False,
            row=1, col=1
        )

    # Row 1, Column 2: Speed Profile Analysis
    fig.add_trace(
        go.Scatter(
            x=lap_df['Distance'], 
            y=lap_df['Speed_kmh'],
            mode='lines',
            line=dict(color=colors['speed'], width=3),
            name='Speed',
            hovertemplate='Distance: %{x:.1f}m<br>Speed: %{y:.1f} km/h<extra></extra>'
        ),
        row=1, col=2
    )

    # Row 1, Column 3: Engine RPM Analysis
    if 'RPM' in lap_df.columns and not lap_df['RPM'].isna().all():
        fig.add_trace(
            go.Scatter(
                x=lap_df['Distance'], 
                y=lap_df['RPM'],
                mode='lines',
                line=dict(color=colors['rpm'], width=3),
                name='RPM',
                hovertemplate='Distance: %{x:.1f}m<br>RPM: %{y:.0f}<extra></extra>'
            ),
            row=1, col=3
        )
    else:
        fig.add_annotation(
            x=0.5, y=0.5, xref='x domain', yref='y domain',
            text="No RPM Data Available", showarrow=False,
            row=1, col=3
        )

    # Row 2, Column 1: Throttle Input Analysis
    fig.add_trace(
        go.Scatter(
            x=lap_df['Distance'], 
            y=lap_df['Throttle'],
            mode='lines',
            line=dict(color=colors['throttle'], width=3),
            name='Throttle',
            hovertemplate='Distance: %{x:.1f}m<br>Throttle: %{y:.3f}<extra></extra>'
        ),
        row=2, col=1
    )

    # Row 2, Column 2: Brake Input Analysis
    fig.add_trace(
        go.Scatter(
            x=lap_df['Distance'], 
            y=lap_df['Brake'],
            mode='lines',
            line=dict(color=colors['brake'], width=3),
            name='Brake',
            hovertemplate='Distance: %{x:.1f}m<br>Brake: %{y:.3f}<extra></extra>'
        ),
        row=2, col=2
    )

    # Row 2, Column 3: Steering Input Analysis
    fig.add_trace(
        go.Scatter(
            x=lap_df['Distance'], 
            y=lap_df['Steering'],
            mode='lines',
            line=dict(color=colors['steering'], width=3),
            name='Steering',
            hovertemplate='Distance: %{x:.1f}m<br>Steering: %{y:.3f}<extra></extra>'
        ),
        row=2, col=3
    )

    # Configure layout and styling
    fig.update_layout(
        title=f"Lap {lap} - Comprehensive Telemetry Analysis ({len(lap_df):,} data points)",
        title_font_size=16,
        height=600,
        showlegend=False,  # Subplot titles provide context
        plot_bgcolor='white',
        paper_bgcolor='white'
    )

    # Configure axis labels and formatting
    for i in range(1, 4):
        fig.update_xaxes(title_text="Distance (m)", row=2, col=i)
        fig.update_xaxes(row=1, col=i, showticklabels=False)  # Clean top row

    # Set descriptive y-axis labels
    fig.update_yaxes(title_text="Gear", row=1, col=1)
    fig.update_yaxes(title_text="Speed (km/h)", row=1, col=2)
    fig.update_yaxes(title_text="RPM", row=1, col=3)
    fig.update_yaxes(title_text="Throttle (0-1)", row=2, col=1)
    fig.update_yaxes(title_text="Brake (0-1)", row=2, col=2)
    fig.update_yaxes(title_text="Steering", row=2, col=3)

    # Add subtle grid for better readability
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

    fig.show()

---

## 5. Interactive Lap Selector (Individual Analysis)

In [25]:
# Interactive lap selector with modern Plotly visualizations
# Provides dynamic lap selection with real-time plot updates

try:
    import ipywidgets as widgets
    from IPython.display import clear_output
    
    # Create interactive slider for lap selection
    lap_selector = widgets.IntSlider(
        value=available_laps[0] if available_laps else 1,
        min=available_laps[0] if available_laps else 1,
        max=available_laps[-1] if available_laps else 1,
        step=1,
        description='Select Lap:',
        style={'description_width': 'initial'}
    )
    out = widgets.Output()

    # Consistent color palette matching the main dashboard
    colors = {
        'gear': '#FF6B35',      # Orange-red for gear progression
        'speed': '#004E89',     # Deep blue for speed profile
        'rpm': '#FF9F1C',       # Golden orange for engine RPM
        'throttle': '#2E8B57',  # Sea green for throttle input
        'brake': '#DC143C',     # Crimson for brake input
        'steering': '#8A2BE2'   # Blue violet for steering input
    }

    def plot_selected(change):
        """
        Callback function for lap selector widget.
        
        Generates individual Plotly plots for each telemetry parameter
        when a new lap is selected. Provides detailed analysis with
        separate plots for better visibility and interaction.
        
        Args:
            change (dict): Widget change event containing new lap selection
        """
        lap = change['new']
        with out:
            clear_output(wait=True)
            lap_df = rows_for_lap(df, lap)
            if lap_df.empty:
                print(f"⚠️  No telemetry data available for Lap {lap}")
                return
            
            print(f"📊 Analyzing Lap {lap} ({len(lap_df):,} data points)\n")
            
            # 1) Gear Progression Analysis
            if 'Gear' in lap_df.columns and not lap_df['Gear'].isna().all():
                fig_gear = go.Figure()
                fig_gear.add_trace(go.Scatter(
                    x=lap_df['Distance'], 
                    y=lap_df['Gear'],
                    mode='lines',
                    line=dict(color=colors['gear'], width=3, shape='hv'),
                    name='Gear',
                    hovertemplate='Distance: %{x:.1f}m<br>Gear: %{y}<extra></extra>'
                ))
                
                # Configure y-axis for integer gear values only
                gear_min = lap_df['Gear'].min()
                gear_max = lap_df['Gear'].max()
                if not (np.isnan(gear_min) or np.isnan(gear_max)):
                    gear_range = list(range(int(gear_min), int(gear_max) + 1))
                    fig_gear.update_yaxes(
                        tickvals=gear_range,
                        range=[gear_min - 0.5, gear_max + 0.5]
                    )
                
                fig_gear.update_layout(
                    title=f'Lap {lap} - Gear Progression Analysis',
                    xaxis_title='Distance (m)',
                    yaxis_title='Gear',
                    height=400,
                    showlegend=False,
                    plot_bgcolor='white',
                    paper_bgcolor='white'
                )
                fig_gear.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
                fig_gear.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
                fig_gear.show()
            else:
                print("⚠️  Gear data not available for this lap")

            # 2) Speed Profile Analysis
            fig_speed = go.Figure()
            fig_speed.add_trace(go.Scatter(
                x=lap_df['Distance'], 
                y=lap_df['Speed_kmh'],
                mode='lines',
                line=dict(color=colors['speed'], width=3),
                name='Speed',
                hovertemplate='Distance: %{x:.1f}m<br>Speed: %{y:.1f} km/h<extra></extra>'
            ))
            fig_speed.update_layout(
                title=f'Lap {lap} - Speed Profile Analysis',
                xaxis_title='Distance (m)',
                yaxis_title='Speed (km/h)',
                height=400,
                showlegend=False,
                plot_bgcolor='white',
                paper_bgcolor='white'
            )
            fig_speed.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
            fig_speed.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
            fig_speed.show()

            # 3) Engine RPM Analysis (if available)
            if 'RPM' in lap_df.columns and not lap_df['RPM'].isna().all():
                fig_rpm = go.Figure()
                fig_rpm.add_trace(go.Scatter(
                    x=lap_df['Distance'], 
                    y=lap_df['RPM'],
                    mode='lines',
                    line=dict(color=colors['rpm'], width=3),
                    name='RPM',
                    hovertemplate='Distance: %{x:.1f}m<br>RPM: %{y:.0f}<extra></extra>'
                ))
                fig_rpm.update_layout(
                    title=f'Lap {lap} - Engine RPM Analysis',
                    xaxis_title='Distance (m)',
                    yaxis_title='RPM',
                    height=400,
                    showlegend=False,
                    plot_bgcolor='white',
                    paper_bgcolor='white'
                )
                fig_rpm.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
                fig_rpm.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
                fig_rpm.show()

            # 4) Throttle Input Analysis (Separate from brake for clarity)
            fig_throttle = go.Figure()
            fig_throttle.add_trace(go.Scatter(
                x=lap_df['Distance'], 
                y=lap_df['Throttle'],
                mode='lines',
                line=dict(color=colors['throttle'], width=3),
                name='Throttle',
                hovertemplate='Distance: %{x:.1f}m<br>Throttle: %{y:.3f}<extra></extra>'
            ))
            fig_throttle.update_layout(
                title=f'Lap {lap} - Throttle Input Analysis',
                xaxis_title='Distance (m)',
                yaxis_title='Throttle Input (0-1)',
                height=400,
                showlegend=False,
                plot_bgcolor='white',
                paper_bgcolor='white'
            )
            fig_throttle.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
            fig_throttle.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
            fig_throttle.show()

            # 5) Brake Input Analysis (Separate for detailed analysis)
            fig_brake = go.Figure()
            fig_brake.add_trace(go.Scatter(
                x=lap_df['Distance'], 
                y=lap_df['Brake'],
                mode='lines',
                line=dict(color=colors['brake'], width=3),
                name='Brake',
                hovertemplate='Distance: %{x:.1f}m<br>Brake: %{y:.3f}<extra></extra>'
            ))
            fig_brake.update_layout(
                title=f'Lap {lap} - Brake Input Analysis',
                xaxis_title='Distance (m)',
                yaxis_title='Brake Input (0-1)',
                height=400,
                showlegend=False,
                plot_bgcolor='white',
                paper_bgcolor='white'
            )
            fig_brake.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
            fig_brake.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
            fig_brake.show()

            # 6) Steering Input Analysis
            fig_steering = go.Figure()
            fig_steering.add_trace(go.Scatter(
                x=lap_df['Distance'], 
                y=lap_df['Steering'],
                mode='lines',
                line=dict(color=colors['steering'], width=3),
                name='Steering',
                hovertemplate='Distance: %{x:.1f}m<br>Steering: %{y:.3f}<extra></extra>'
            ))
            fig_steering.update_layout(
                title=f'Lap {lap} - Steering Input Analysis',
                xaxis_title='Distance (m)',
                yaxis_title='Steering Angle',
                height=400,
                showlegend=False,
                plot_bgcolor='white',
                paper_bgcolor='white'
            )
            fig_steering.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
            fig_steering.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
            fig_steering.show()

    # Set up widget event handling and display
    lap_selector.observe(plot_selected, names='value')
    display(lap_selector, out)
    
    # Generate initial plot for default lap selection
    plot_selected({'new': lap_selector.value})
    
except Exception as e:
    print("⚠️  Interactive widgets not available or failed to initialize:")
    print(f"   Error: {e}")
    print("\n💡 Fallback: Use Section 4 (Comprehensive Dashboard) to view all laps sequentially.")
    print("   Or install ipywidgets: pip install ipywidgets")

IntSlider(value=1, description='Select Lap:', max=2, min=1, style=SliderStyle(description_width='initial'))

Output()

---

## 6. Data Export and Persistence

### 6.1 Investigate the distance issue before exporting

In [26]:
# Investigate distance issue before exporting
print("🔍 Distance columns investigation:")
print(f"Available columns related to distance:")
distance_cols = [col for col in df.columns if 'distance' in col.lower() or 'traveled' in col.lower()]
print(f"  {distance_cols}")

print(f"\n📊 Analysis per lap:")
for lap in available_laps:
    lap_df = rows_for_lap(df, lap)
    if not lap_df.empty:
        print(f"\nLap {lap}:")
        print(f"  Distance: {lap_df['Distance'].min():.1f} - {lap_df['Distance'].max():.1f} m")
        if 'DistanceTraveled_m' in lap_df.columns:
            print(f"  DistanceTraveled_m: {lap_df['DistanceTraveled_m'].min():.1f} - {lap_df['DistanceTraveled_m'].max():.1f} m")
        if 'DistanceTraveled_m_Original' in lap_df.columns:
            print(f"  DistanceTraveled_m_Original: {lap_df['DistanceTraveled_m_Original'].min():.1f} - {lap_df['DistanceTraveled_m_Original'].max():.1f} m")
        
        # Check if Distance is actually normalized
        first_distance = lap_df['Distance'].iloc[0]
        last_distance = lap_df['Distance'].iloc[-1]
        print(f"  Starts from ~0?: {first_distance < 100}")
        print(f"  Lap length: {last_distance - first_distance:.1f} m")


🔍 Distance columns investigation:
Available columns related to distance:
  ['DistanceTraveled_m', 'Distance', 'DistanceTraveled_m_Original']

📊 Analysis per lap:

Lap 1:
  Distance: 0.0 - 6431.5 m
  DistanceTraveled_m: 146.8 - 6578.4 m
  DistanceTraveled_m_Original: 146.8 - 6578.4 m
  Starts from ~0?: True
  Lap length: 6431.5 m

Lap 2:
  Distance: 0.0 - 5748.6 m
  DistanceTraveled_m: 6586.7 - 12335.3 m
  DistanceTraveled_m_Original: 6586.7 - 12335.3 m
  Starts from ~0?: True
  Lap length: 5748.6 m


### 6.2 Exporting the individual lap data

In [27]:
# Export individual lap data to separate CSV files for detailed analysis
# This enables focused analysis of specific laps and external processing

out_dir = Path("../TELEMETRY/LAPS_OUTPUT/")
out_dir.mkdir(exist_ok=True)

exported_count = 0
for lap in available_laps:
    rows = rows_for_lap(df, lap)
    if not rows.empty:
        # Create clean export with normalized distance only
        export_df = rows.copy()
        
        # Remove absolute distance columns to avoid confusion
        columns_to_remove = []
        if 'DistanceTraveled_m' in export_df.columns:
            columns_to_remove.append('DistanceTraveled_m')
        if 'DistanceTraveled_m_Original' in export_df.columns:
            columns_to_remove.append('DistanceTraveled_m_Original')
        
        export_df = export_df.drop(columns=columns_to_remove)
        
        # Ensure Distance column is at the front for clarity
        if 'Distance' in export_df.columns:
            cols = ['Distance'] + [col for col in export_df.columns if col != 'Distance']
            export_df = export_df[cols]
        
        filename = out_dir / f"lap_{lap}_telemetry.csv"
        export_df.to_csv(filename, index=False)
        exported_count += 1
        
        # Verify the exported data
        distance_range = f"{export_df['Distance'].min():.1f} - {export_df['Distance'].max():.1f}m"
        print(f"✅ Exported Lap {lap}: {len(rows):,} data points → {filename}")
        print(f"   📏 Normalized distance range: {distance_range}")



✅ Exported Lap 1: 934 data points → ..\TELEMETRY\LAPS_OUTPUT\lap_1_telemetry.csv
   📏 Normalized distance range: 0.0 - 6431.5m
✅ Exported Lap 2: 814 data points → ..\TELEMETRY\LAPS_OUTPUT\lap_2_telemetry.csv
   📏 Normalized distance range: 0.0 - 5748.6m


#### 6.2.1 Export Summary

In [28]:
print(f"\n📊 Export Summary:")
print(f"   Total laps exported: {exported_count}")
print(f"   Output directory: {out_dir.absolute()}")
print(f"   ✅ Using normalized distance (0-based per lap)")
print(f"   ❌ Absolute distance columns removed to avoid confusion")
print(f"\n   Files can be used for:")
print(f"   • Individual lap performance analysis")
print(f"   • Machine learning model training")
print(f"   • External data visualization tools")
print(f"   • Comparative lap analysis")


📊 Export Summary:
   Total laps exported: 2
   Output directory: c:\Users\victo\Desktop\Documents\Cuarto Año\Primer Cuatrimestre\F1_AC_Digital_Twin\data-analytics\..\TELEMETRY\LAPS_OUTPUT
   ✅ Using normalized distance (0-based per lap)
   ❌ Absolute distance columns removed to avoid confusion

   Files can be used for:
   • Individual lap performance analysis
   • Machine learning model training
   • External data visualization tools
   • Comparative lap analysis


---